In [163]:
import pandas as pd
import numpy as np
import json

In [160]:
# need this!
# !pip install geocoder

    100% |████████████████████████████████| 71kB 524kB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 503kB/s ta 0:00:011
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [161]:
import geocoder

In [164]:
df = pd.read_csv("Crowd Estimates, 1.21.2017 - Sheet1.csv", skiprows=6,  header=None, thousands=",")
col_names = ["place", "state", "country", "estimate_low", "estimate_high", "est3", "not121", "source", "source2", "source3"]
df.columns = col_names
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 663 entries, 0 to 662
Data columns (total 10 columns):
place            663 non-null object
state            544 non-null object
country          661 non-null object
estimate_low     596 non-null float64
estimate_high    597 non-null float64
est3             5 non-null float64
not121           5 non-null float64
source           608 non-null object
source2          173 non-null object
source3          21 non-null object
dtypes: float64(4), object(6)
memory usage: 51.9+ KB


In [165]:
# a few added here to fix data in spreadsheet
state_lookup = { 'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa', 'AZ': 'Arizona', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'GU': 'Guam', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota', 'MO': 'Missouri', 'MP': 'Northern Mariana Islands', 'MS': 'Mississippi', 'MT': 'Montana', 'NA': 'National', 'NC': 'North Carolina', 'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'PR': 'Puerto Rico', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia', 'VI': 'Virgin Islands', 'VT': 'Vermont', 'WA': 'Washington', 'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming', 'CA (Seacrest)':'California', 'Guam': 'Guam'}

In [166]:
# drop what we don't care about
df.drop(["est3", "not121"], axis=1, inplace=True)
# drop rows that lack a max estimate
df.dropna(axis=0, how='any', subset=['estimate_high'], inplace=True)
df

,place,state,country,estimate_low,estimate_high,source,source2,source3
0,"Abilene, TX",TX,US,200.0,200.0,http://www.reporternews.com/story/news/local/2...,NaN,NaN
1,"Accident, MD",MD,US,54.0,54.0,Twitter; on-site witness,NaN,NaN
2,"Adak, AK",AK,US,10.0,10.0,adn.com,NaN,NaN
3,"Adrian, MI",MI,US,130.0,150.0,https://www.facebook.com/events/847360115406578/,http://www.lenconnect.com/news/20170122/marche...,NaN
4,"Ajo, AZ",AZ,US,250.0,250.0,https://www.facebook.com/plugins/post.php?href...,NaN,NaN
5,"Alameda, CA",CA,US,8.0,8.0,FB,NaN,NaN
6,"Alamosa, CO",CO,US,350.0,350.0,https://twitter.com/zepol/status/8232349739352...,NaN,NaN
7,"Albany, CA",CA,US,500.0,500.0,https://twitter.com/ben_silverman/status/82303...,Albany PD (reported via google form),NaN
8,"Albany, NY",NY,US,7000.0,9000.0,http://www.timesunion.com/local/article/Albany...,FB,NaN
9,"Alexandria, VA",VA,US,17.0,17.0,https://twitter.com/TinaPStacy/status/82322604...,NaN,NaN


In [167]:
# this is spelled wrong
t_town = df[ df['place']=='Tblisi, Georgia']
df.loc[t_town.index,'place'] = 'Tbilisi, Georgia'

# this is that header type row
df.drop( df[ df.place.str.contains('partial')].index, axis=0, inplace=True )

In [168]:

def fix_us_place_names(place, country):
    if country != "US":
        return place
    res  = place.split(', ')
    if len(res) < 2 or res[-1] not in state_lookup:
        return place
    state = state_lookup[res[-1]]
    res[-1]= state
    res.append('United States')
    return ', '.join(res)

# z = "Abilene, TX"
# # z = "Encinitas, CA (Seacrest)"
# country = "US"
# fix_us_place_names(z, country)

# df['col'] = 'str' + df['col'].astype(str)
# df["new_column"] = df.apply(lambda x: x["A"] * x["B"], axis = 1)
# df.columns = df.columns.append('full_place')

# create a new col with the full place name, for easier geocoding
df = df.assign(full_place=df.place)
df.full_place = df.apply(lambda x: fix_us_place_names(x.place, x.country) , axis=1 )



In [220]:
# g = geocoder.google("New York, New York, United States")
# g.latlng
cache = {}


In [242]:
# make our new cols
df = df.assign(latlng=df.place)
df = df.assign(lng=df.place)


def get_lat(x): 
    if x in cache:
        return cache[x][0]    
    print(x)
    g = geocoder.google(x)
    if g.latlng:
        cache[x] = g.latlng
        return g.latlng[0]
    else:
        return pd.np.nan

def get_lng(x): 
    if x in cache:
        return cache[x][1]
    print(x)
    g = geocoder.google(x)
    if g.latlng:
        cache[x] = g.latlng        
        return g.latlng[1]
    else:
        return pd.np.nan

df['lat'] = df.full_place.apply(lambda x: get_lat(x) )
df['lng'] = df.full_place.apply(lambda x: get_lng(x) )
df
# this can actually be run multiple times, which will fix missing data

Milhelm, Pennsylvania, United States
Milhelm, Pennsylvania, United States


,place,state,country,estimate_low,estimate_high,source,source2,source3,full_place,lat,lng,latlng
0,"Abilene, TX",TX,US,200.0,200.0,http://www.reporternews.com/story/news/local/2...,NaN,NaN,"Abilene, Texas, United States",32.448736,-99.733144,"Abilene, TX"
1,"Accident, MD",MD,US,54.0,54.0,Twitter; on-site witness,NaN,NaN,"Accident, Maryland, United States",39.628696,-79.319759,"Accident, MD"
2,"Adak, AK",AK,US,10.0,10.0,adn.com,NaN,NaN,"Adak, Alaska, United States",51.880000,-176.658056,"Adak, AK"
3,"Adrian, MI",MI,US,130.0,150.0,https://www.facebook.com/events/847360115406578/,http://www.lenconnect.com/news/20170122/marche...,NaN,"Adrian, Michigan, United States",41.897547,-84.037166,"Adrian, MI"
4,"Ajo, AZ",AZ,US,250.0,250.0,https://www.facebook.com/plugins/post.php?href...,NaN,NaN,"Ajo, Arizona, United States",32.371725,-112.860710,"Ajo, AZ"
5,"Alameda, CA",CA,US,8.0,8.0,FB,NaN,NaN,"Alameda, California, United States",37.765206,-122.241636,"Alameda, CA"
6,"Alamosa, CO",CO,US,350.0,350.0,https://twitter.com/zepol/status/8232349739352...,NaN,NaN,"Alamosa, Colorado, United States",37.469449,-105.870021,"Alamosa, CO"
7,"Albany, CA",CA,US,500.0,500.0,https://twitter.com/ben_silverman/status/82303...,Albany PD (reported via google form),NaN,"Albany, California, United States",37.886870,-122.297747,"Albany, CA"
8,"Albany, NY",NY,US,7000.0,9000.0,http://www.timesunion.com/local/article/Albany...,FB,NaN,"Albany, New York, United States",42.652579,-73.756232,"Albany, NY"
9,"Alexandria, VA",VA,US,17.0,17.0,https://twitter.com/TinaPStacy/status/82322604...,NaN,NaN,"Alexandria, Virginia, United States",38.804836,-77.046921,"Alexandria, VA"


In [245]:
df.dropna(axis=0, how='any', subset=['lat', 'lng'], inplace=True)
# df.drop(["latlng"], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 595 entries, 0 to 662
Data columns (total 12 columns):
place            595 non-null object
state            503 non-null object
country          595 non-null object
estimate_low     594 non-null float64
estimate_high    595 non-null float64
source           592 non-null object
source2          173 non-null object
source3          21 non-null object
full_place       595 non-null object
lat              595 non-null float64
lng              595 non-null float64
latlng           595 non-null object
dtypes: float64(4), object(8)
memory usage: 60.4+ KB


In [246]:
df.to_json("marches.json", orient="records")

In [236]:

# df.info()
# cache['Madison, Wisconsin, United States']
# df [ df['full_place']== "Madison, Wisconsin, United States" ]
# df[df.lat == False]
# df[df.lng == False]

,place,state,country,estimate_low,estimate_high,source,source2,source3,full_place,lat,lng
0,"Abilene, TX",TX,US,200.0,200.0,http://www.reporternews.com/story/news/local/2...,NaN,NaN,"Abilene, Texas, United States",32.4487,-99.7331
1,"Accident, MD",MD,US,54.0,54.0,Twitter; on-site witness,NaN,NaN,"Accident, Maryland, United States",39.6287,-79.3198
2,"Adak, AK",AK,US,10.0,10.0,adn.com,NaN,NaN,"Adak, Alaska, United States",51.88,-176.658
3,"Adrian, MI",MI,US,130.0,150.0,https://www.facebook.com/events/847360115406578/,http://www.lenconnect.com/news/20170122/marche...,NaN,"Adrian, Michigan, United States",41.8975,-84.0372
4,"Ajo, AZ",AZ,US,250.0,250.0,https://www.facebook.com/plugins/post.php?href...,NaN,NaN,"Ajo, Arizona, United States",32.3717,-112.861
5,"Alameda, CA",CA,US,8.0,8.0,FB,NaN,NaN,"Alameda, California, United States",37.7652,-122.242
6,"Alamosa, CO",CO,US,350.0,350.0,https://twitter.com/zepol/status/8232349739352...,NaN,NaN,"Alamosa, Colorado, United States",37.4694,-105.87
7,"Albany, CA",CA,US,500.0,500.0,https://twitter.com/ben_silverman/status/82303...,Albany PD (reported via google form),NaN,"Albany, California, United States",37.8869,-122.298
8,"Albany, NY",NY,US,7000.0,9000.0,http://www.timesunion.com/local/article/Albany...,FB,NaN,"Albany, New York, United States",42.6526,-73.7562
9,"Alexandria, VA",VA,US,17.0,17.0,https://twitter.com/TinaPStacy/status/82322604...,NaN,NaN,"Alexandria, Virginia, United States",38.8048,-77.0469
